# Building a Part of Speech Tagger with Keras

This notebook is based on nlpforhackers' post: https://nlpforhackers.io/lstm-pos-tagger-keras/. We will look at how to build a part of speech tagger using a LSTM layer.

We will use NLTK's treebank corpus. This corpus has, among other kinds of information, annotations about the parts of speech of the words.

In [2]:
import nltk
nltk.download('treebank')

tagged_sentences = nltk.corpus.treebank.tagged_sents()
 
print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words: 100676


As always, before training a model, we need to split the data in training and testing data:

In [6]:
import random
random.seed(1234)
tagged_sentences = list(tagged_sentences) # we convert the data to a list
random.shuffle(tagged_sentences)

In [9]:
threshold = int(len(tagged_sentences)*.6)
train = tagged_sentences[:threshold]
test = tagged_sentences[threshold:]

Let’s restructure the data a bit. Let’s separate the words from the tags.

In [10]:
import numpy as np
 
train_sentences, train_tags =[], [] 
for tagged_sentence in train:
    sentence, tags = zip(*tagged_sentence)
    train_sentences.append(np.array(sentence))
    train_tags.append(np.array(tags))
    
test_sentences, test_tags =[], [] 
for tagged_sentence in test:
    sentence, tags = zip(*tagged_sentence)
    test_sentences.append(np.array(sentence))
    test_tags.append(np.array(tags))

In [11]:
print(train_sentences[5])
print(train_tags[5])

['They' 'are' 'keeping' 'a' 'close' 'watch' 'on' 'the' 'yield' 'on' 'the'
 'S&P' '500' '.']
['PRP' 'VBP' 'VBG' 'DT' 'JJ' 'NN' 'IN' 'DT' 'NN' 'IN' 'DT' 'NNP' 'CD' '.']


Let's now convert the words and tags to indices. We will reserve index 0 for padding, and index 1 for words that are out of the vocabulary (**OOV - Out of Vocabulary**).

In [14]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding
tag2index['-OOV-'] = 1  # There may also be unknown tags in the test data

In [15]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
 
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    s_int = []
    for t in s:
        try:
            s_int.append(tag2index[t])
        except KeyError:
            s_int.append(tag2index['-OOV-'])
            
    test_tags_y.append(s_int)
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[754, 6164, 7542, 7659, 7464, 102, 8463, 8244]
[4138, 8525, 3112, 4960, 4849, 1, 1147, 439, 7282, 1408, 5386, 439, 1, 1592, 7789, 7464, 6913, 8095, 439, 8621, 5896, 1, 8244]
[11, 15, 9, 29, 7, 43, 43, 45]
[41, 29, 4, 20, 29, 4, 31, 38, 35, 41, 4, 38, 4, 31, 4, 7, 10, 4, 38, 31, 42, 4, 45]


Since Keras can only deal with fixed size sequences, we will pad all sequences to fit the longest sentence in the training data.

In [16]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)

271


In [17]:
from keras.preprocessing.sequence import pad_sequences
 
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

Using TensorFlow backend.


[ 754 6164 7542 7659 7464  102 8463 8244    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

We can now define the model. 
A simple approach to build a part-of-speech tagger in Keras is to stack a dense layer at the output of each RNN cell. This is achieved with `TimeDistributed`. Note  how the output of the model is now a list of sequences of vectors. Each vector will contain the probability of each tag. In particular:

* We will introduce an embedding layer.
* THe LSTM layer will have the parameter `return_sequences=True` so that we have access to the entire sequence.
* The output of each LSTM cell will have a `Dense` layer. We can do this using the `TimeDistributed` layer.

In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 271, 128)          1111424   
_________________________________________________________________
lstm_1 (LSTM)                (None, 271, 256)          394240    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 271, 47)           12079     
_________________________________________________________________
activation_1 (Activation)    (None, 271, 47)           0         
Total params: 1,517,743
Trainable params: 1,517,743
Non-trainable params: 0
_________________________________________________________________


We also need to convert the labels into one-hot encoding. Since the input are sequences of labels we cannot use Keras' `to_categorical` so we define our own function:

In [19]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [20]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


We can finally train the model:

In [22]:
history = model.fit(train_sentences_X, cat_train_tags_y, batch_size=128, epochs=40, validation_split=0.2)
 

Train on 1878 samples, validate on 470 samples
Epoch 1/40
1878/1878 [==============================] - 21s 11ms/step - loss: 1.7629 - acc: 0.8446 - val_loss: 0.6247 - val_acc: 0.9036
Epoch 2/40
1878/1878 [==============================] - 21s 11ms/step - loss: 0.6393 - acc: 0.8875 - val_loss: 0.4856 - val_acc: 0.9038
Epoch 3/40
1878/1878 [==============================] - 21s 11ms/step - loss: 0.4455 - acc: 0.9062 - val_loss: 0.4177 - val_acc: 0.9057
Epoch 4/40
1878/1878 [==============================] - 19s 10ms/step - loss: 0.3990 - acc: 0.9086 - val_loss: 0.3997 - val_acc: 0.9047
Epoch 5/40
1878/1878 [==============================] - 19s 10ms/step - loss: 0.3768 - acc: 0.9077 - val_loss: 0.3772 - val_acc: 0.9060
Epoch 6/40
1878/1878 [==============================] - 19s 10ms/step - loss: 0.3619 - acc: 0.9078 - val_loss: 0.3599 - val_acc: 0.9058
Epoch 7/40
1878/1878 [==============================] - 23s 12ms/step - loss: 0.3536 - acc: 0.9079 - val_loss: 0.3536 - val_acc: 0.9066
E

If the results look too good, it's because they are too good! We need to ignore the classification of the padded symbols, but let's leave it aside for now. Below is an evaluation using the test data.

In [23]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")

1566/1566 [==============================] - 6s 4ms/step
acc: 93.28064533333517


Let's find the PoS of two test sentences:

In [24]:
test_samples = [
    "running is very important for me .".split(),
    "I was running every day for a month .".split()
]
print(test_samples)

[['running', 'is', 'very', 'important', 'for', 'me', '.'], ['I', 'was', 'running', 'every', 'day', 'for', 'a', 'month', '.']]


These test sentences need to be vectorised:

In [25]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[5416 3046  879  225 7049 5468 8244    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

And the predictions are ...

In [26]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[4.4955608e-02 1.5484265e-02 2.2056568e-02 ... 1.8686477e-02
   4.4350747e-02 1.8839989e-02]
  [6.1708014e-02 1.3672160e-02 2.2489037e-02 ... 1.5864795e-02
   4.0098306e-02 1.4878591e-02]
  [9.4220236e-02 1.0509712e-02 2.2016199e-02 ... 1.2349819e-02
   4.6605412e-02 1.0682364e-02]
  ...
  [9.9996805e-01 1.2200499e-10 7.6720987e-08 ... 7.1617540e-10
   3.5734286e-06 3.6779796e-10]
  [9.9996805e-01 1.2200499e-10 7.6720987e-08 ... 7.1617540e-10
   3.5734286e-06 3.6779796e-10]
  [9.9996805e-01 1.2200523e-10 7.6721136e-08 ... 7.1617401e-10
   3.5734354e-06 3.6780007e-10]]

 [[1.9523820e-02 2.0028079e-02 2.1672221e-02 ... 1.8764738e-02
   1.5450116e-02 1.7119510e-02]
  [2.2560960e-02 1.6599175e-02 2.3360679e-02 ... 1.4789831e-02
   1.2849574e-02 1.0460691e-02]
  [4.8549145e-02 1.2866377e-02 2.6165031e-02 ... 1.4345896e-02
   3.7202392e-02 1.0042745e-02]
  ...
  [9.9996805e-01 1.2200499e-10 7.6720987e-08 ... 7.1617540e-10
   3.5734286e-06 3.6779796e-10]
  [9.9996805e-01 1.2200499e-10 7.672

These predictions show, for each word, the probabilities assigned to each possible label. Let's choose the label with the highest probability (using numpy's `argmax`) and then convert from label index to label:

In [27]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [28]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['-PAD-', '-PAD-', '-PAD-', '-PAD-', 'IN', 'NN', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 

As expected, the padding symbol is over-used. To solve this problem we can define a custom accuracy metric that ignores the paddings. Remember that the index for the padding symbol is 0:

In [29]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

Let's now retrain using this new accuracy metric:

In [39]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 271, 128)          1111424   
_________________________________________________________________
lstm_4 (LSTM)                (None, 271, 256)          394240    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 271, 47)           12079     
_________________________________________________________________
activation_4 (Activation)    (None, 271, 47)           0         
Total params: 1,517,743
Trainable params: 1,517,743
Non-trainable params: 0
_________________________________________________________________


In [40]:
from keras.callbacks import EarlyStopping
history = model.fit(train_sentences_X, cat_train_tags_y, batch_size=128, epochs=100, validation_split=0.2) #,
#                    callbacks=[EarlyStopping(monitor='val_ignore_accuracy')])

Train on 1878 samples, validate on 470 samples
Epoch 1/100
1878/1878 [==============================] - 20s 10ms/step - loss: 1.7266 - acc: 0.8440 - ignore_accuracy: 0.0018 - val_loss: 0.6099 - val_acc: 0.9036 - val_ignore_accuracy: 0.0000e+00
Epoch 2/100
1878/1878 [==============================] - 20s 11ms/step - loss: 0.6572 - acc: 0.8780 - ignore_accuracy: 0.0543 - val_loss: 0.4371 - val_acc: 0.9037 - val_ignore_accuracy: 0.1100
Epoch 3/100
1878/1878 [==============================] - 24s 13ms/step - loss: 0.3991 - acc: 0.9061 - ignore_accuracy: 0.1153 - val_loss: 0.3948 - val_acc: 0.9042 - val_ignore_accuracy: 0.1263
Epoch 4/100
1878/1878 [==============================] - 18s 10ms/step - loss: 0.3802 - acc: 0.9062 - ignore_accuracy: 0.1187 - val_loss: 0.3920 - val_acc: 0.9039 - val_ignore_accuracy: 0.1097
Epoch 5/100
1878/1878 [==============================] - 21s 11ms/step - loss: 0.3695 - acc: 0.9062 - ignore_accuracy: 0.1177 - val_loss: 0.3739 - val_acc: 0.9040 - val_ignore_a

1878/1878 [==============================] - 20s 11ms/step - loss: 0.1064 - acc: 0.9747 - ignore_accuracy: 0.7338 - val_loss: 0.1251 - val_acc: 0.9701 - val_ignore_accuracy: 0.6944
Epoch 44/100
1878/1878 [==============================] - 19s 10ms/step - loss: 0.0968 - acc: 0.9776 - ignore_accuracy: 0.7643 - val_loss: 0.1171 - val_acc: 0.9723 - val_ignore_accuracy: 0.7166
Epoch 45/100
1878/1878 [==============================] - 20s 11ms/step - loss: 0.0882 - acc: 0.9801 - ignore_accuracy: 0.7901 - val_loss: 0.1089 - val_acc: 0.9744 - val_ignore_accuracy: 0.7366
Epoch 46/100
1878/1878 [==============================] - 20s 11ms/step - loss: 0.0802 - acc: 0.9822 - ignore_accuracy: 0.8130 - val_loss: 0.1024 - val_acc: 0.9761 - val_ignore_accuracy: 0.7551
Epoch 47/100
1878/1878 [==============================] - 20s 11ms/step - loss: 0.0728 - acc: 0.9844 - ignore_accuracy: 0.8359 - val_loss: 0.0963 - val_acc: 0.9776 - val_ignore_accuracy: 0.7710
Epoch 48/100
1878/1878 [===================

In [41]:
predictions = model.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['VBG', 'VBZ', 'RB', 'JJ', 'IN', 'PRP', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', 

We can see the great difference in accuracy when we ignore the padding symbol. Also, note that the results reported by the original post https://nlpforhackers.io/lstm-pos-tagger-keras/ are better, probably because they used two LSTM chains: one going forwards, and another going backwards. This is what is called a **bidirectional recurrent network**, and it usually reports better results because each prediction is based on the context on the left and the right of the token.